In [17]:
from py.Training import training_and_testing
from py.vae import NeuralNet as nnet
import py.config as CFG

In [11]:
import os
import glob
from tqdm import tqdm

import numpy as np

import torch
import torchvision.transforms.functional as TF
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import random_split

from matplotlib import pyplot as plt
plt.rc('font', family='NanumGothic')
from PIL import Image

In [12]:
def rotate_images(image):
    # 이미지를 0도, 90도, 180도, 270도로 회전시키는 함수
    images = []
    for angle in [0, 90, 180, 270]:
        rotated_image = TF.rotate(image, angle)
        images.append(rotated_image)
    return images

def gauss_noise(image_tensor, sigma=0.05):
    # 이미지에 가우시안 노이즈를 추가하는 함수
    noise = torch.randn(image_tensor.size()) * sigma
    noisy_image = image_tensor + noise
    noisy_image = torch.clamp(noisy_image, 0, 1)
    return noisy_image


class VAECustomDataset(Dataset):
    def __init__(self, file_paths, transform=None, gauss_sigma=0.05):
        # 데이터셋 초기화
        self.file_paths = file_paths
        self.transform = transform
        self.gauss_sigma = gauss_sigma

    def __len__(self):
        # 데이터셋의 길이를 반환
        return len(self.file_paths)

    def __getitem__(self, idx):
        # 주어진 인덱스에 해당하는 데이터를 반환
        image = Image.open(self.file_paths[idx])
        if self.transform:
            # 변환이 주어진 경우
            original_image = self.transform(image)  # 원본 이미지 변환
            noisy_image = gauss_noise(original_image, self.gauss_sigma)  # 노이즈 추가 이미지 생성
            images = rotate_images(image)  # 회전 이미지 생성
            transformed_images = [self.transform(img) for img in images]  # 회전 이미지를 변환
            noisy_images = [gauss_noise(img, self.gauss_sigma) for img in transformed_images]  # 회전된 이미지에 노이즈 추가
            return original_image, noisy_image, transformed_images, noisy_images
        else:
            # 변환이 주어지지 않은 경우 원본 이미지 반환
            return image


In [14]:
# 데이터 폴더 경로 설정
folder = r"D:\PPJ\Model\data\Normal"
file_path = glob.glob(os.path.join(folder, "*.jpg"))

In [15]:
train_size = int(0.8 * len(file_path))
test_size = len(file_path) - train_size

train_paths, test_paths = random_split(file_path, [train_size, test_size])

train_dataset = VAECustomDataset(file_paths=train_paths, transform=CFG.transform)
test_dataset = VAECustomDataset(file_paths=test_paths, transform=CFG.transform)

In [19]:
neuralnet = nnet(height=CFG.height, width=CFG.width, channel=CFG.channel,
                 device=CFG.device, ngpu=CFG.ngpu, ksize=CFG.ksize, z_dim=CFG.z_dim)

Encoder(
  (encoder_conv): Sequential(
    (0): Conv2d(1, 16, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
    (1): ELU(alpha=1.0)
    (2): Conv2d(16, 16, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
    (3): ELU(alpha=1.0)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(16, 32, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
    (6): ELU(alpha=1.0)
    (7): Conv2d(32, 32, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
    (8): ELU(alpha=1.0)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(32, 64, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
    (11): ELU(alpha=1.0)
    (12): Conv2d(64, 64, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
    (13): ELU(alpha=1.0)
  )
  (encoder_dense): Sequential(
    (0): Flatten()
    (1): Linear(in_features=921600, out_features=512, bias=True)
    (2): ELU(alpha=1.0)
    (3): Linear(in_features=512, out_features=40, bia

In [ ]:
training_and_testing(train_dataset=train_dataset, test_dataset= test_dataset,
                      epochs=CFG.epochs, batch_size=CFG.batch_size)